# Looking at the correlation between cloud fraction and the calculated feedback parameters

### Add paths to run locally or off NIRD

In [11]:
import sys
# Add common resources folder to path
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/Common/")
sys.path.append("/mnt/mcc-ns9600k/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/")
sys.path.append("/home/jonahks/git_repos/netcdf_analysis/Common/")

from imports import (
    pd, np, xr, mpl, plt, sns, os, 
    datetime, sys, crt, gridspec,
    ccrs, metrics, Iterable
    )

from functions import (
    masked_average, add_weights, sp_map,
    season_mean, get_dpm, leap_year, share_ylims,
    to_png, mute_ax
    )

from classes import SatComp_Metric, CT_SLF_Metric
from collections import deque
%matplotlib inline

## Load feedback specific functions

In [12]:
from feedback_functions import *

### Check running location and adjust working directory appropriately.

In [13]:
host = os.uname()[1]
if 'jupyter' in host.split('-'): # Check if running on NIRD through the Jupyter Hub
    print('Running through MC2 Jupyter Hub')
    model_dir = '/mnt/mcc-ns9600k/jonahks/'
    os.chdir(model_dir)

else:  # Assume that we're running on a local machine and mounting NIRD
    print('Running on %s, attempting to mount ns9600k/jonahks/ from NIRD' % str(host))
    os.system('fusermount -zu ~/drivemount/')  # unmount first
    os.system('sshfs jonahks@login.nird.sigma2.no:"p/jonahks/" ~/drivemount/')    # Calling mountnird from .bashrc doesn't work
    os.chdir('/home/jonahks/drivemount/')
    save_dir = '~/DATAOUT/'
    save_to = os.path.expanduser(save_dir)

output_dir = 'figures/'
case_dir = 'satcomp/'   # inconsistent label compared to jupy_test
conv_dir ='convectivephase/'

# Check that each important directory can be accessed:    
access_paths = os.path.exists(output_dir) and os.path.exists(case_dir) and os.path.exists(conv_dir)
print('Can access all directory paths:', access_paths)

Running through MC2 Jupyter Hub
Can access all directory paths: True


In [4]:
fb_dir ='feedbacks_soden/mod_by_me/output_jonahks/'

## Calculate GOCCP "present-day" total cloud fraction

Get GOCCP value to calculate bias

In [25]:
_goccp_metric = SatComp_Metric(casedir=case_dir)

Loading GOCCP data...done.
Loading CALIOP SLFs...done
Loading CERES-EBAF fluxes...done.


In [26]:
goccp_data = _goccp_metric.goccp_data

cldtot_monthly = add_weights(goccp_data['CLDTOT_CAL'].groupby('time.month').mean('time'))

# cldtot_monthly
mask = cldtot_monthly['lat'] < 66
arc_mean = masked_average(cldtot_monthly,mask=mask,weights=cldtot_monthly['cell_weight'],dim=['lat','lon'])

Get variables

__LW Feedback Parameter__

__Winter Cloud from the Models__

In [7]:
init_dir = 'satcomp/'
pert_dir = 'satcomp_+4K/'

In [8]:
# Testing something to order cases and label them.
model_pairs = {}
model_pairs['CAM6_satcomp_SFCnonorm_wbf_1_inp_1'] = ['20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1','20200817_4Ksst_cam6satcomp_wbf_1_inp_1']
model_pairs['CESM2_satcomp_SFCnonorm_wbf_1_inp_1'] = ['CESM2_slfvars','20201231_4Ksst_cesm2satcomp_wbf_1_inp_1']
model_pairs['CAM61_satcomp_SFCnonorm_wbf_1_inp_1'] = ['20200414_205148_singleparam_cam61satcomp_wbf_1_inp_1','20200817_4Ksst_cam61satcomp_wbf_1_inp_1']
model_pairs['CAM6_satcomp_SFCnonorm_wbf_1.25_inp_10'] = ['20200512_013308_fitting_runs_cam6satcomp16_wbf_1.25_inp_10','20200817_4Ksst_cam6satcomp_wbf_1.25_inp_10']
model_pairs['CAM61_satcomp_SFCnonorm_wbf_0.5_inp_0.05'] = ['20200629_morn_cam61satcomp_wbf_0.5_inp_0.05','20200817_4Ksst_cam61satcomp_wbf_0.5_inp_0.05']
model_pairs['CAM61_satcomp_SFCnonorm_wbf_0.2_inp_0.1'] = ['20200512_012745_fitting_runs_cam61satcomp_wbf_0.2_inp_0.1','20200817_4Ksst_cam61satcomp_wbf_0.2_inp_0.1']
model_pairs['CESM2_satcomp_SFCnonorm_wbf_1_inp_100'] = ['20200713_CESM2_satcomp_wbf_1_inp_100','20201231_4Ksst_CESM2_satcomp_wbf_1_inp_100']

### Grab total cloud and low cloud from the present-day simulations.

In [9]:
totcloud_init = {}
lowcloud_init = {}

for i in model_pairs:
    print(i)
    _pair = model_pairs[i]
    _init_path = '%s%s/%s.nc' % (init_dir,_pair[0],_pair[0])
    
    _initall = xr.open_dataset(_init_path)
    _temp_dat = _initall['CLDTOT_CAL'].sel(time=slice('2010-02-01','2013-01-01'))#.groupby('time.season').mean('time')
    totcloud_init[i] = _temp_dat
    _temp_dat = _initall['CLDLOW_CAL'].sel(time=slice('2010-02-01','2013-01-01'))#.groupby('time.season').mean('time')
    lowcloud_init[i] = _temp_dat
    
    _initall.close()

CAM6_satcomp_SFCnonorm_wbf_1_inp_1
CESM2_satcomp_SFCnonorm_wbf_1_inp_1
CAM61_satcomp_SFCnonorm_wbf_1_inp_1
CAM6_satcomp_SFCnonorm_wbf_1.25_inp_10
CAM61_satcomp_SFCnonorm_wbf_0.5_inp_0.05
CAM61_satcomp_SFCnonorm_wbf_0.2_inp_0.1
CESM2_satcomp_SFCnonorm_wbf_1_inp_100


### Calculate the average model cloud in april and (feb,mar,apr)

In [10]:
apr_totcloud_dict = {}
fma_totcloud_dict = {}
for i in totcloud_init:
    _ds = add_weights(totcloud_init[i])
    _apr_mean = _ds.groupby('time.month').mean('time').sel(month=4)
    _fma_mean = _ds.groupby('time.month').mean('time').sel(month=[2,3,4]).mean('month')
    mask = _ds['lat'] < 66
    _apr_avg = masked_average(_apr_mean,dim=['lat','lon'],weights=_apr_mean['cell_weight'],mask=mask)
    _fma_avg = masked_average(_fma_mean,dim=['lat','lon'],weights=_fma_mean['cell_weight'],mask=mask)
    apr_totcloud_dict[i] = _apr_avg
    fma_totcloud_dict[i] = _fma_avg
    
apr_totcloud = np.array(list(apr_totcloud_dict.values())).flatten()
fma_totcloud = np.array(list(fma_totcloud_dict.values())).flatten()

/opt/conda/lib/python3.7/site-packages/xarray/core/common.py:664: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  self, group, squeeze=squeeze, restore_coord_dims=restore_coord_dims
/opt/conda/lib/python3.7/site-packages/xarray/core/common.py:664: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  self, group, squeeze=squeeze, restore_coord_dims=restore_coord_dims
/opt/conda/lib/python3.7/site-packages/xarray/core/common.py:664: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, the dimension order of these coordinates will be restored as well unless you specify restore_coord_dims=False.
  self, group, squeeze=squeeze, restore_coord_dims=restore_coord_dims
/opt

In [ ]:
# Do correlation
y = np.array(out2['cloudlw_fb'])
x = np.array(apr_totcloud) - float(arc_mean.sel(month=4))

model = LinearRegression()
model.fit(x.reshape(-1,1), y.reshape(-1,1))
# x_new = np.linspace(min(x), max(x), 100)
x_new = np.linspace(min(x), 0, 100)
y_new = model.predict(x_new[:, np.newaxis])
y_pred = model.predict(x.reshape(-1,1))
categories = np.array([0, 1, 2, 3, 4, 5, 6, 7])
colormap = np.array(sns.color_palette('colorblind'))

# Plot correlation
axs[3].scatter(x, y, c=colormap[categories],marker='o')
axs[3].plot(x_new,y_new,c='grey',linestyle='dotted')

In [14]:
og_cesm = xr.open_dataset('satcomp/CESM2_slfvars/atm/hist/CESM2_slfvars.cam.h0.2009-03.nc')
og_noresm = xr.open_dataset('satcomp/20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1/atm/hist/20200504_145018_fitting_runs_cam6satcomp_wbf_1_inp_1.cam.h0.2009-03.nc')

In [17]:
og_cesm.coords

Coordinates:
  * lat             (lat) float64 -90.0 -88.11 -86.21 ... 86.21 88.11 90.0
  * lon             (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * lev             (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev            (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * isotherms_mpc   (isotherms_mpc) float64 233.1 238.1 243.1 ... 268.1 273.1
  * cosp_prs        (cosp_prs) float64 9e+04 7.4e+04 6.2e+04 ... 2.45e+04 9e+03
  * cosp_tau        (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 100.0
  * cosp_scol       (cosp_scol) int32 1 2 3 4 5 6 7 8 9 10
  * cosp_ht         (cosp_ht) float64 1.896e+04 1.848e+04 ... 720.0 240.0
  * cosp_sr         (cosp_sr) float64 -0.495 0.605 2.1 4.0 ... 55.0 70.0 539.5
  * cosp_sza        (cosp_sza) float64 0.0 20.0 40.0 60.0 80.0
  * cosp_dbze       (cosp_dbze) float64 -72.5 -42.5 -37.5 ... 12.5 17.5 50.0
  * cosp_htmisr     (cosp_htmisr) float64 0.0 250.0 750.0 ... 1.6e+04 1.8e+04
  * cosp_tau

In [18]:
og_cesm.variables

AttributeError: 'Frozen' object has no attribute 'long_name'